In [2]:
from google.colab import drive 
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [69]:
import re
import os
import cv2
import numpy as np

In [ ]:
!unzip /content/gdrive/MyDrive/SFU/results.zip -d /content/gdrive/MyDrive/SFU/results

In [70]:
def assinge_view(filename):
  top_pattern = ".*_top"
  front_pattern = ".*_front"
  side_pattern = ".*_side"

  if re.match(top_pattern,filename) is not None:
    arms,back,legs,seat,view = re.match(top_pattern,filename)[0].split("_")
    return arms,back,legs,seat,view
  elif re.match(front_pattern,filename) is not None:
    arms,back,legs,seat,view = re.match(front_pattern,filename)[0].split("_")
    return arms,back,legs,seat,view
  elif re.match(side_pattern,filename) is not None: 
    arms,back,legs,seat,view = re.match(side_pattern,filename)[0].split("_")
    return arms,back,legs,seat,view
  else:
    print("Invali filename")
    return None

def prepare_scorer_input(folder,dimension):

  imagesTop = []
  imagesSide = []
  imagesFront = []
  imagesTop_ids = []
  imagesSide_ids = []
  imagesFront_ids = []
  length = len(os.listdir(folder)) // 3

  # sorted_list = sorted(os.listdir(folder),key =sort_key )

  for filename in os.listdir(folder):
    
    arms,back,legs,seat,view = assinge_view(filename)
  
    if view == None:
      continue
    img = cv2.imread(folder+filename)
    img = cv2.resize(img, dsize=(dimension, dimension), interpolation=cv2.INTER_CUBIC)
    img = np.nan_to_num(img)

    if img is not None:
      if view == "side":
        imagesSide.append(1. - img / 255.)
        imagesSide_ids.append([arms,back,legs,seat])
      elif view == "top":
        imagesTop.append(1. - img / 255.)
        imagesTop_ids.append([arms,back,legs,seat])
      elif view =="front":
        imagesFront.append(1. - img / 255.)
        imagesFront_ids.append([arms,back,legs,seat])
      else:
        print("view not found!")

  return imagesTop,imagesFront,imagesSide, imagesTop_ids,imagesFront_ids,imagesSide_ids

In [71]:
import tensorflow as tf
from tensorflow import  keras
def load_model(path):
  return keras.models.load_model(path)

In [106]:
from statistics import mean
def sort_score(top_model,front_model,side_model,folder,dim):
  imagesTop,imagesFront,imagesSide, imagesTop_ids,imagesFront_ids,imagesSide_ids = prepare_scorer_input(folder,dim)
  score = list()
  score_all = list()
  sorted_ids = list()


  for i in range(len(imagesTop)):
    top_image = np.expand_dims(imagesTop[i], axis=0)
    front_image = np.expand_dims(imagesFront[i], axis=0)
    side_image = np.expand_dims(imagesSide[i], axis=0)
    top_pred = top_model.predict(top_image)
    front_pred = front_model.predict(front_image)
    side_pred = side_model.predict(side_image)
    score_all.append([top_pred[0][0],front_pred[0][0],side_pred[0][0]])
    score.append(min([top_pred[0][0],front_pred[0][0],side_pred[0][0]]))
  
  print(score)
  score_index = np.argsort(score)[::-1]
  score.sort(reverse=True)
  
  print(score_index)
  print(score)
  for i in range(len(score_index)):
    sorted_ids.append(imagesSide_ids[score_index[i]])
    print(imagesSide_ids[score_index[i]])
  return sorted_ids


In [73]:
top_model =  load_model("/content/gdrive/MyDrive/SFU/best_top_aug.h5")
front_model = load_model("/content/gdrive/MyDrive/SFU/best_front_aug.h5")
side_model = load_model("/content/gdrive/MyDrive/SFU/best_side_aug.h5")

## Generated chair ids from best to worst from Set A

In [113]:
folder = "/content/gdrive/MyDrive/SFU/results/results/set_a/"
sorted_ids = np.array(sort_score(top_model,front_model,side_model,folder,56))
file_name = "/content/gdrive/MyDrive/SFU/"+"set_a_results.txt"
np.savetxt(file_name,sorted_ids,fmt='%s')


[0.33217242, 0.36583266, 0.22452141, 0.69931453, 0.17381737, 0.528092, 0.16485998, 0.40322337, 0.09736527, 0.74858224, 0.3265466, 0.5191375, 0.16485998, 0.1289561, 0.33217242, 0.08680094, 0.12876807, 0.24476716, 0.22452141, 0.17381737]
[ 9  3  5 11  7  1  0 14 10 17 18  2  4 19 12  6 13 16  8 15]
[0.74858224, 0.69931453, 0.528092, 0.5191375, 0.40322337, 0.36583266, 0.33217242, 0.33217242, 0.3265466, 0.24476716, 0.22452141, 0.22452141, 0.17381737, 0.17381737, 0.16485998, 0.16485998, 0.1289561, 0.12876807, 0.09736527, 0.08680094]
['2585', '2323', '43872', '43872']
['2323', '2585', '43872', '43872']
['2323', '43872', '2585', '43872']
['2585', '43872', '2323', '43872']
['2585', '2323', '2323', '43872']
['2323', '2585', '2585', '43872']
['2323', '2323', '2585', '43872']
['43872', '2323', '2585', '43872']
['2585', '43872', '2323', '2585']
['43872', '2585', '2585', '2323']
['43872', '2585', '43872', '2323']
['2323', '2585', '43872', '2323']
['2323', '43872', '2585', '2323']
['43872', '43872',

## Generated chair ids from best to worst from Set B

In [114]:
folder = "/content/gdrive/MyDrive/SFU/results/results/set_b/"
sorted_ids = np.array(sort_score(top_model,front_model,side_model,folder,56))
file_name = "/content/gdrive/MyDrive/SFU/"+"set_b_results.txt"
np.savetxt(file_name,sorted_ids,fmt='%s')

[0.53808403, 0.4715122, 0.18379752, 0.17663279, 0.43022814, 0.46182021, 0.11683808, 0.14549206, 0.30462158, 0.065704145, 0.73517805, 0.5294633, 0.052787587, 0.6237442, 0.050890047, 0.40722635, 0.6887615, 0.15066788, 0.38436583, 0.38841194, 0.32984328, 0.7053486, 0.05386973, 0.27571514, 0.63537014, 0.3283648, 0.29503697, 0.5335865, 0.068366945, 0.19420819, 0.30448696, 0.044767104, 0.22308184, 0.73910797, 0.33513892, 0.06410516, 0.24932472, 0.055800386, 0.045277823, 0.06423748, 0.14723745, 0.37562263, 0.044140648, 0.13344485, 0.17445128, 0.007913601, 0.017042689, 0.04160759, 0.026859708, 0.070482664]
[33 10 21 16 24 13  0 27 11  1  5  4 15 19 18 41 34 20 25  8 30 26 23 36
 32 29  2  3 44 17 40  7 43  6 49 28  9 39 35 37 22 12 14 38 31 42 47 48
 46 45]
[0.73910797, 0.73517805, 0.7053486, 0.6887615, 0.63537014, 0.6237442, 0.53808403, 0.5335865, 0.5294633, 0.4715122, 0.46182021, 0.43022814, 0.40722635, 0.38841194, 0.38436583, 0.37562263, 0.33513892, 0.32984328, 0.3283648, 0.30462158, 0.3044

## Generated chair ids from best to worst from Set C

In [115]:
folder = "/content/gdrive/MyDrive/SFU/results/results/set_c/"
sorted_ids = np.array(sort_score(top_model,front_model,side_model,folder,56))
file_name = "/content/gdrive/MyDrive/SFU/"+"set_c_results.txt"
np.savetxt(file_name,sorted_ids,fmt='%s')

[0.025989579, 0.11724077, 0.05069646, 0.0003119489, 0.040339265, 0.0010794449, 0.083432876, 0.004619932, 0.0025592707, 0.026043827, 0.033210278, 0.04353044, 0.06264931, 0.0021460329, 0.026955653, 0.13547745, 0.0010691183, 0.0023103182, 0.01556708, 0.2609175, 0.01180489, 0.016104287, 0.43746704, 0.0128013985, 0.2033082, 0.27772254, 0.0020327505, 0.1195737, 0.075228184, 0.0022387265, 0.04381529, 0.020895507, 0.076800294, 0.025354484, 0.0111496635, 0.0063188598, 0.026397439, 0.13845468, 0.09741463, 0.15540932, 0.042511694, 0.048580624, 0.000700375, 0.010392462, 0.03119459, 0.05754206, 0.044646647, 0.14712965, 0.046303272, 0.8739376, 0.019164495, 0.013428735, 0.005805272, 0.0068240804, 0.05519645, 0.0053422223, 0.0373014, 0.0293468, 0.0078300405, 0.0062927487, 0.006090942, 0.019585527, 0.011553455, 0.08799209, 0.0035812838, 0.043469235, 0.17060956, 0.060263246, 0.0062927487, 0.03759319, 0.009082951, 0.009924961, 0.14392495, 0.039648842, 0.49134868, 0.4480594, 0.06967119, 0.06688976, 0.0344